# Sequence+ Visualizations & Evaluation

***Author***: Jensen Holm

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import polars as pl
from IPython.display import Video
from utils import get_savant_video

DATA_DIR = os.path.join("..", "data")

## Best Two Pitch Combo's by Sequence+

In [2]:
combo_df = pl.read_parquet(
    os.path.join(DATA_DIR, "mean_sequence+_combo_leaderboard.parquet")
)

combo_df.head()

pitcher,player_name,combo,n,Mean Sequence+,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,…,botCmd FS,botOvr,botStf,botCmd,botxRV100,Stf+ CH,Loc+ CH,Pit+ CH,Stf+ CU,Loc+ CU,Pit+ CU,Stf+ FA,Loc+ FA,Pit+ FA,Stf+ SI,Loc+ SI,Pit+ SI,Stf+ SL,Loc+ SL,Pit+ SL,Stf+ KC,Loc+ KC,Pit+ KC,Stf+ FC,Loc+ FC,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO,first_name,last_name
i64,str,str,u32,f64,i64,i64,str,str,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,str,str
543243,"""Gray, Sonny""","""SI-SI""",94,106.516822,12768,2024,"""Sonny Gray""","""STL""",34,13,9,3.8,3.84,28,28,0,0,0,0,166.1,671,142,79,71,21,39,0,3,7,1,203,176,147,93,11,835,…,null,62,58,54,-0.61,69.0,91.0,86.0,119.0,96.0,97.0,107.0,97.0,101.0,93.0,114.0,112.0,152.0,114.0,125.0,null,null,null,92.0,98.0,96.0,null,null,null,110,103,105,null,null,null,"""Sonny""","""Gray"""
543243,"""Gray, Sonny""","""FF-ST""",72,105.851609,12768,2024,"""Sonny Gray""","""STL""",34,13,9,3.8,3.84,28,28,0,0,0,0,166.1,671,142,79,71,21,39,0,3,7,1,203,176,147,93,11,835,…,null,62,58,54,-0.61,69.0,91.0,86.0,119.0,96.0,97.0,107.0,97.0,101.0,93.0,114.0,112.0,152.0,114.0,125.0,null,null,null,92.0,98.0,96.0,null,null,null,110,103,105,null,null,null,"""Sonny""","""Gray"""
592332,"""Gausman, Kevin""","""FF-FF""",869,105.252293,14107,2024,"""Kevin Gausman""","""TOR""",33,14,11,2.9,3.83,31,31,2,1,0,0,181.0,756,165,86,77,20,56,0,2,3,2,162,204,225,104,13,1032,…,60.0,55,50,59,-0.18,null,null,null,null,null,null,99.0,105.0,105.0,90.0,102.0,94.0,88.0,90.0,95.0,null,null,null,null,null,null,88.0,108.0,106.0,94,105,104,null,null,null,"""Kevin""","""Gausman"""
669923,"""Kirby, George""","""SI-FF""",225,105.137735,25436,2024,"""George Kirby""","""SEA""",26,14,11,4.2,3.53,33,33,0,0,0,0,191.0,778,181,82,75,22,23,0,7,5,1,179,232,229,101,25,937,…,49.0,61,53,62,-0.54,null,null,null,null,null,null,102.0,107.0,101.0,106.0,108.0,108.0,110.0,104.0,110.0,114.0,110.0,113.0,97.0,99.0,93.0,109.0,100.0,108.0,106,106,106,null,null,null,"""George""","""Kirby"""
676710,"""Crawford, Kutter""","""FF-FS""",94,104.95429,20531,2024,"""Kutter Crawford""","""BOS""",28,9,16,1.9,4.36,33,33,0,0,0,0,183.2,759,155,97,89,34,51,0,9,4,0,175,171,256,91,37,911,…,48.0,58,45,58,-0.34,null,null,null,null,null,null,109.0,109.0,112.0,null,null,null,100.0,99.0,102.0,98.0,97.0,99.0,114.0,104.0,104.0,87.0,102.0,106.0,108,105,107,null,null,null,"""Kutter""","""Crawford"""


In [3]:
# right now, not interested in two pitch combos of the same pitch.
# this is an issue that I want to look into in the future. The model should ideally
# not be awarding seqeunces with pitches of the same type a high number.
combo_df = combo_df.filter(
    pl.col("combo").str.split("-").list.get(0) != 
    pl.col("combo").str.split("-").list.get(1)
)

combo_df.head()

pitcher,player_name,combo,n,Mean Sequence+,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,GS,CG,ShO,SV,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,GB,FB,LD,IFFB,Balls,…,botCmd FS,botOvr,botStf,botCmd,botxRV100,Stf+ CH,Loc+ CH,Pit+ CH,Stf+ CU,Loc+ CU,Pit+ CU,Stf+ FA,Loc+ FA,Pit+ FA,Stf+ SI,Loc+ SI,Pit+ SI,Stf+ SL,Loc+ SL,Pit+ SL,Stf+ KC,Loc+ KC,Pit+ KC,Stf+ FC,Loc+ FC,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO,first_name,last_name
i64,str,str,u32,f64,i64,i64,str,str,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,str,str
543243,"""Gray, Sonny""","""FF-ST""",72,105.851609,12768,2024,"""Sonny Gray""","""STL""",34,13,9,3.8,3.84,28,28,0,0,0,0,166.1,671,142,79,71,21,39,0,3,7,1,203,176,147,93,11,835,…,null,62,58,54,-0.61,69.0,91.0,86.0,119.0,96.0,97.0,107.0,97.0,101.0,93.0,114.0,112.0,152.0,114.0,125.0,null,null,null,92.0,98.0,96.0,null,null,null,110,103,105,null,null,null,"""Sonny""","""Gray"""
669923,"""Kirby, George""","""SI-FF""",225,105.137735,25436,2024,"""George Kirby""","""SEA""",26,14,11,4.2,3.53,33,33,0,0,0,0,191.0,778,181,82,75,22,23,0,7,5,1,179,232,229,101,25,937,…,49.0,61,53,62,-0.54,null,null,null,null,null,null,102.0,107.0,101.0,106.0,108.0,108.0,110.0,104.0,110.0,114.0,110.0,113.0,97.0,99.0,93.0,109.0,100.0,108.0,106,106,106,null,null,null,"""George""","""Kirby"""
676710,"""Crawford, Kutter""","""FF-FS""",94,104.95429,20531,2024,"""Kutter Crawford""","""BOS""",28,9,16,1.9,4.36,33,33,0,0,0,0,183.2,759,155,97,89,34,51,0,9,4,0,175,171,256,91,37,911,…,48.0,58,45,58,-0.34,null,null,null,null,null,null,109.0,109.0,112.0,null,null,null,100.0,99.0,102.0,98.0,97.0,99.0,114.0,104.0,104.0,87.0,102.0,106.0,108,105,107,null,null,null,"""Kutter""","""Crawford"""
682243,"""Miller, Bryce""","""ST-FS""",54,104.699123,29837,2024,"""Bryce Miller""","""SEA""",25,12,8,2.8,2.94,31,31,0,0,0,0,180.1,704,131,62,59,21,45,1,3,2,0,171,184,213,86,16,959,…,56.0,61,54,54,-0.55,null,null,null,null,null,null,124.0,102.0,106.0,100.0,102.0,99.0,122.0,96.0,101.0,92.0,104.0,111.0,93.0,80.0,85.0,95.0,107.0,111.0,112,101,105,null,null,null,"""Bryce""","""Miller"""
592791,"""Taillon, Jameson""","""FC-ST""",98,104.49659,11674,2024,"""Jameson Taillon""","""CHC""",32,12,8,2.3,3.27,28,28,0,0,0,0,165.1,675,154,67,60,21,33,0,1,4,2,125,204,206,99,14,833,…,null,56,41,65,-0.24,78.0,102.0,95.0,104.0,111.0,113.0,66.0,106.0,96.0,74.0,104.0,92.0,104.0,116.0,120.0,null,null,null,92.0,103.0,98.0,null,null,null,85,107,102,null,null,null,"""Jameson""","""Taillon"""


In [11]:
# I want to take a look at some of these sequences by sonny gray and george kirby
gray_miller_24_df = (pl.scan_parquet(os.path.join(DATA_DIR, "pitches_sequence+.parquet"))
                     .with_columns(
                        (pl.col("pitch_type") + "-" + pl.col("prev_pitch_type")).alias("combo")
                     )
                     .filter(
                        pl.col("game_date").dt.year() == 2024,
                         pl.col("player_name").is_in(["Kirby, George"]) &
                         pl.col("combo").is_in(["SL-FF"]),
                     )
                     .sort("Sequence+", descending=True)
                     .collect())

pitches = gray_miller_24_df.slice(0, 1)
prev_pitches = pitches.select(
    "game_pk",
    "inning",
    "at_bat_number",
    "pitch_number",
)

_ = get_savant_video(plays=pitches)
_ = get_savant_video(plays=prev_pitches.with_columns(
    (pl.col("pitch_number") - 1).alias("pitch_number")
))

print(pitches.select("Sequence+").item())

100%|██████████| 2/2 [00:02<00:00,  1.21s/it]

117.0591306264227


In [6]:
# I want to take a look at some of these sequences by sonny gray and george kirby
gray_miller_24_df = (pl.scan_parquet(os.path.join(DATA_DIR, "pitches_sequence+.parquet"))
                     .with_columns(
                        (pl.col("pitch_type") + "-" + pl.col("prev_pitch_type")).alias("combo")
                     )
                     .filter(
                        pl.col("game_date").dt.year() == 2024,
                         pl.col("player_name").is_in(["Gray, Sonny"]) &
                         pl.col("combo").is_in(["FF-ST"]),
                     )
                     .sort("Sequence+", descending=True)
                     .collect())

pitches = gray_miller_24_df.slice(0, 1)
prev_pitches = pitches.select(
    "game_pk",
    "inning",
    "at_bat_number",
    "pitch_number",
)

_ = get_savant_video(plays=pitches)
_ = get_savant_video(plays=prev_pitches.with_columns(
    (pl.col("pitch_number") - 1).alias("pitch_number")
))

print(pitches.select("Sequence+").item())

100%|██████████| 2/2 [00:00<00:00,  3.15it/s]

115.28702374069077


## Video of best sequences by Sequence+